In [1]:
import pandas as pd
import os
import numpy as np
from brat_parser import get_entities_relations_attributes_groups
import gzip
import math

In [2]:
admissions_df =  pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/ADMISSIONS.csv')

In [3]:
subjects_df = pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/PATIENTS.csv')

In [4]:
admissions_df = admissions_df.merge(subjects_df[['SUBJECT_ID','DOB','GENDER']], on='SUBJECT_ID')

In [5]:
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df['DOB'] = pd.to_datetime(admissions_df['DOB'])
admissions_df['DEATHTIME'] = pd.to_datetime(admissions_df['DEATHTIME'])
admissions_df['DISCHTIME'] = pd.to_datetime(admissions_df['DISCHTIME'])


In [6]:
def get_age(row):
    try:
        return (row.ADMITTIME-row.DOB) / np.timedelta64(1, 's')/ 60./60/24/365
    except:
        return np.nan
admissions_df['AGE'] = admissions_df.apply(get_age, axis=1)

In [7]:
os.listdir('../../../LHC_mimic/mimic3_1.4/raw_data/')

['D_ICD_PROCEDURES.csv',
 'INPUTEVENTS_CV.csv',
 'CAREGIVERS.csv.gz',
 'CPTEVENTS.csv',
 'DATETIMEEVENTS.csv',
 'PROCEDURES_ICD.csv.gz',
 'D_ICD_DIAGNOSES.csv.gz',
 'CHARTEVENTS.csv.gz',
 'DRGCODES.csv.gz',
 'PROCEDUREEVENTS_MV.csv',
 'D_ITEMS.csv',
 'MICROBIOLOGYEVENTS.csv.gz',
 'ICUSTAYS.csv.gz',
 'CALLOUT.csv.gz',
 'MICROBIOLOGYEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PROCEDURES_ICD.csv',
 'D_ICD_DIAGNOSES.csv',
 'PRESCRIPTIONS.csv.gz',
 'PROCEDUREEVENTS_MV.csv.gz',
 'CALLOUT.csv',
 'TRANSFERS.csv',
 'ADMISSIONS.csv',
 'DIAGNOSES_ICD.csv.gz',
 'INPUTEVENTS_MV.csv.gz',
 'DATETIMEEVENTS.csv.gz',
 'PATIENTS.csv',
 'CPTEVENTS.csv.gz',
 'NOTEEVENTS.csv.gz',
 'PATIENTS.csv.gz',
 'LABEVENTS.csv',
 'PRESCRIPTIONS.csv',
 'SERVICES.csv.gz',
 'DRGCODES.csv',
 'D_LABITEMS.csv',
 'NOTEEVENTS.csv',
 'D_ICD_PROCEDURES.csv.gz',
 'D_ITEMS.csv.gz',
 'TRANSFERS.csv.gz',
 'robots.txt',
 'ICUSTAYS.csv',
 'CHARTEVENTS.csv',
 'D_CPT.csv.gz',
 'CAREGIVERS.csv',
 'INPUTEVENTS_CV.csv.gz',
 'LABEVENTS.csv.gz',
 'D

In [8]:
patients_df = pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/PATIENTS.csv')

In [9]:
patients_df['DOD'] = pd.to_datetime(patients_df['DOD'])


In [10]:
patients_df


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaT,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaT,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaT,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaT,NaN,NaN,0
...,...,...,...,...,...,...,...,...
46515,31840,44089,M,2026-05-25 00:00:00,NaT,NaN,NaN,0
46516,31841,44115,F,2124-07-27 00:00:00,NaT,NaN,NaN,0
46517,31842,44123,F,2049-11-26 00:00:00,2135-01-12,2135-01-12 00:00:00,NaN,1
46518,31843,44126,F,2076-07-25 00:00:00,NaT,NaN,NaN,0


In [11]:
mortality_subset = patients_df[patients_df['DOD'].apply(lambda x: (not pd.isnull(x)))]

In [12]:
strict_mortality_subset = admissions_df.merge(mortality_subset[['SUBJECT_ID','DOD']], on='SUBJECT_ID', how='outer')
# strict_mortality_subset = mortality_subset

In [13]:
strict_mortality_subset['disch_to_death'] = strict_mortality_subset.apply(lambda row:  ( (row['DOD']-row['DISCHTIME']) / np.timedelta64(1, 's')/ 60./60 )  ,axis=1)


In [14]:
len(strict_mortality_subset[strict_mortality_subset['disch_to_death']>0]['SUBJECT_ID'].unique())#/len(strict_mortality_subset['SUBJECT_ID'].unique())

11336

In [15]:
len(strict_mortality_subset)
# len(admissions_df)

58976

In [16]:
strict_mortality_subset = strict_mortality_subset[strict_mortality_subset.apply(lambda row:  row['disch_to_death']>=24 and row['disch_to_death']<=720  ,axis=1)]

In [17]:
strict_mortality_subset

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOB,GENDER,AGE,DOD,disch_to_death
42,476,370,123421,2153-03-04 01:17:00,2153-03-08 18:57:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,...,2153-03-03 14:34:00,2153-03-03 23:55:00,PNEUMONIA,0,1,2069-08-26,F,83.575489,2153-03-12,77.050000
72,70,68,108329,2174-01-04 22:21:00,2174-01-19 11:30:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2174-01-04 17:40:00,2174-01-05 03:00:00,WEAKNESS,0,1,2132-02-29,F,41.879264,2174-02-11,540.500000
108,106,103,130744,2144-08-12 17:37:00,2144-08-20 11:15:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,NaN,NaN,SUBARACHNOID HEMORRHAGE,0,1,2084-04-14,F,60.369134,2144-09-01,276.750000
150,148,109,155726,2142-08-13 04:03:00,2142-08-16 18:17:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,...,2142-08-13 01:49:00,2142-08-13 05:59:00,DYSPNEA,0,1,2117-08-07,F,25.033339,2142-08-30,317.716667
185,199,149,154869,2135-02-18 19:16:00,2135-02-26 17:00:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,...,2135-02-18 09:11:00,2135-02-18 13:00:00,CORONARY ARTERY DISEASE,0,1,1835-02-18,F,NaN,2135-03-02,79.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58788,58926,99836,101018,2116-05-21 19:23:00,2116-07-17 16:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOSPICE-MEDICAL FACILITY,Medicare,...,NaN,NaN,HEPATITIS,0,1,2064-04-13,F,52.139199,2116-07-23,127.500000
58832,57000,93844,188035,2144-06-11 16:37:00,2144-06-17 18:34:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,...,2144-06-11 12:59:00,2144-06-11 21:45:00,ABDOMINAL PAIN,0,1,2101-06-02,F,43.056691,2144-07-11,557.433333
58836,57761,96234,195310,2117-12-29 18:21:00,2118-01-01 16:06:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,2117-12-29 14:04:00,2117-12-29 21:27:00,UPPER GI BLEED,0,1,2055-08-12,M,62.424013,2118-01-31,703.900000
58900,57491,95380,198194,2196-10-28 21:19:00,2196-11-09 17:40:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,...,NaN,NaN,KIDNEY FAILURE,0,1,2145-02-02,M,51.772296,2196-11-23,318.333333


In [18]:
# procedures_df = pd.read_csv('../../LHC_mimic/mimic3_1.4/raw_data/PROCEDURES_ICD.csv')

In [19]:
# diagnoses = pd.read_csv('../../LHC_mimic/mimic3_1.4/raw_data/DIAGNOSES_ICD.csv')

In [20]:
notes_df = pd.read_csv('../../../LHC_mimic/mimic3_1.4/raw_data/NOTEEVENTS.csv')

/gpfs/gsfs12/users/moldwinar/chiqa-fsl-ks/tr3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
notes_df=notes_df.dropna(subset=['HADM_ID','SUBJECT_ID'], how='any')

In [22]:
notes_df = notes_df.merge(admissions_df[['HADM_ID','AGE','GENDER']], on='HADM_ID')

In [23]:
notes_df = notes_df[notes_df['AGE']>15]

In [24]:
notes_df['HADM_ID'] = notes_df['HADM_ID'].apply(lambda x: int(x))

In [25]:
notes_df['CHARTTIME']= pd.to_datetime(notes_df['CHARTTIME'])

In [26]:
sections_df = pd.read_csv('../section_annotations_df.csv')

In [27]:
adm_sections = [ 'chief_complaint','history_of_present_illness', 'ros', 'assessment_and_plan','allergies','past_medical_history','family_history','social_history','physical_exam']

In [28]:
merged = notes_df.merge(sections_df[['ROW_ID','sections']],on='ROW_ID')

In [29]:
merged['has_adm_section']=merged['sections'].apply(lambda x: any([ y in x for y in adm_sections]))

In [30]:
# admission_notes =merged[merged.apply(lambda row: row['has_adm_section'] and row['CATEGORY'] in ['Nursing/other','Nursing','General','Physician '],axis=1)].sort_values(by='CHARTTIME')
admission_notes =merged[merged.apply(lambda row: 'discharge' in row['TEXT'].lower() and row['CATEGORY'] in ['Nursing/other','Nursing','General','Physician '],axis=1)].sort_values(by='CHARTTIME')

In [31]:
# admission_notes

In [32]:
admission_notes['CATEGORY'].unique()

array(['Nursing/other', 'Nursing', 'Physician ', 'General'], dtype=object)

In [33]:
mortality_subjects = list(mortality_subset['SUBJECT_ID'])
# mortality_admissions = list(mortality_subset['HADM_ID'])


negative_admission_notes = notes_df[notes_df.apply(lambda row:(not row['SUBJECT_ID'] in mortality_subjects) ,axis=1)]

In [34]:
negative_admission_notes=negative_admission_notes[negative_admission_notes['CATEGORY']=='Discharge summary']

In [35]:
negative_admission_notes = negative_admission_notes.sample(frac=1)

In [36]:
len(negative_admission_notes['HADM_ID'].unique())
len(negative_admission_notes['HADM_ID'])

# len(negative_admission_notes['SUBJECT_ID'].unique())

29771

In [37]:
negative_admission_notes = negative_admission_notes.drop_duplicates(subset =['SUBJECT_ID'], keep='first' )
negative_admission_notes = negative_admission_notes.sort_values(by='CHARTTIME', ascending = True)
negative_admission_notes = negative_admission_notes.drop_duplicates(subset =['HADM_ID'], keep='first' )

In [38]:
len(negative_admission_notes)

21812

In [39]:
strict_mortality_admissions = list(strict_mortality_subset['HADM_ID'])

positive_admission_notes = notes_df[notes_df.apply(lambda row:(row['HADM_ID'] in strict_mortality_admissions) ,axis=1)]


In [40]:
len(positive_admission_notes)

72219

In [41]:
positive_admission_notes=positive_admission_notes[positive_admission_notes['CATEGORY']=='Discharge summary']

In [42]:
# strict_mortality_subset['HADM_ID']
# 161773 in list(admission_notes['HADM_ID'])
# admission_notes['HADM_ID']
# # len(positive_admission_notes)
# strict_mortality_subset['SUBJECT_ID']
# 98669 in admission_notes['SUBJECT_ID']
len(positive_admission_notes)

2044

In [43]:
positive_admission_notes = positive_admission_notes.sort_values(by='CHARTTIME', ascending = False)
positive_admission_notes = positive_admission_notes.drop_duplicates(subset=['HADM_ID'],keep='first')
positive_admission_notes = positive_admission_notes.drop_duplicates(subset=['SUBJECT_ID'],keep='first')

In [44]:
len(positive_admission_notes)

1688

In [45]:
positive_admission_notes

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,AGE,GENDER
212,178,26880,135453,2162-03-25,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,82.215841,M
375,227,10044,124073,2152-10-11,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-10-2**] Dischar...,81.695023,F
1489,218,11018,131769,2154-03-28,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2154-3-24**] ...,69.275449,M
1514,221,75347,152030,2174-06-03,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2174-5-3**] D...,64.221476,M
2762,36,5458,130840,2137-03-16,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2137-3-7**] D...,76.461914,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815691,53342,16804,102306,2192-03-15,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2192-3-2**] Discharg...,84.041414,F
1816932,53607,61998,176304,2200-03-28,NaT,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2200-3-25**] ...,43.901714,M
1817192,56811,2896,178124,2105-11-02,NaT,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname **], [**Known firstna...",88.602314,M
1817520,57780,28848,137939,2168-10-26,NaT,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 12352**],[**Known fir...",88.567452,M


In [46]:
set(positive_admission_notes['SUBJECT_ID']).intersection(set(negative_admission_notes['SUBJECT_ID']))

set()

In [47]:
# len(pd.Series(mortality_admissions).unique())/len(admissions_df['HADM_ID'].unique())
len(positive_admission_notes)/(len(negative_admission_notes)+len(positive_admission_notes))
# len(positive_admission_notes['SUBJECT_ID'].unique())/(len(negative_admission_notes['SUBJECT_ID'].unique())+len(positive_admission_notes['SUBJECT_ID'].unique()))

0.07182978723404256

In [48]:
pd.read_csv('../../notes_benchmark/data/in-hospital-mortality/filtered_train_listfile.csv')['y_true'].mean()

0.12821677263094156

In [49]:
positive_admission_notes['label']=1
negative_admission_notes['label']=0

In [50]:
# #for sampling distribution of large dataframe to match smaller df
# def get_same_distribution_sample(smaller,larger, q, column):
#     smaller['age_quantile'],bins = pd.qcut(cad_admission_notes[column],q=q, retbins=True)
#     larger['age_quantile'] = pd.cut(larger[column], bins=bins)
#     larger=larger.dropna(subset=['age_quantile'])
#     larger_quantile_size = larger.groupby('age_quantile').apply(len).min()
#     bins = larger['age_quantile'].unique()
#     all_quantiles = []
#     for quantile in bins:
#         print(quantile)
#         all_quantiles.append(larger[larger['age_quantile']==quantile].sample(larger_quantile_size))
#     return pd.concat(all_quantiles)

In [51]:
# negatives_age_sampled = get_same_distribution_sample(cad_admission_notes,negative_admission_notes,6,'AGE')

In [52]:
# shuffled_age_gender_negatives = pd.concat([negatives_age_sampled[negatives_age_sampled['GENDER']=='F'].sample(frac=0.5),(negatives_age_sampled[negatives_age_sampled['GENDER']=='M'])]).sample(frac=1)

In [53]:
# shuffled_age_gender_negatives.groupby('GENDER').apply(len).max()/shuffled_age_gender_negatives.groupby('GENDER').apply(len).min()

In [54]:
# len(cad_admission_notes)

In [55]:
# cad_admission_notes['label']=1
# shuffled_age_gender_negatives['label']=0

In [56]:
train_and_test_data = pd.concat([positive_admission_notes[['TEXT','label']],negative_admission_notes[['TEXT','label']]]).sample(frac=1)

In [57]:
# from sklearn.model_selection import train_test_split

# train_patients, test_patients = train_test_split(train_and_test_data['SUBJECT_ID'], test_size=0.1)
# train_patients,val_patients = train_test_split(train_patients, test_size=0.05)

from sklearn.model_selection import train_test_split

train, test = train_test_split(train_and_test_data, test_size=0.1)
train,val = train_test_split(train, test_size=0.05)

In [58]:
train.to_csv('../data/mortality_train.csv')
val.to_csv('../data/mortality_val.csv')
test.to_csv('../data/mortality_test.csv')

In [2]:
import pandas as pd
val = pd.read_csv('../data/mortality_val.csv')#['label'].mean()

In [11]:
from transformers import DistilBertTokenizer

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [13]:
tokenizer.encode('this is a snentence')

[101, 2023, 2003, 1037, 1055, 21576, 10127, 102]

In [15]:
val['TEXT'].apply(tokenizer.encode).apply(len).mean()

2981.758034026465

In [1]:
import pandas as pd
all_data = pd.concat([pd.read_csv('../data/mortality_train.csv'),pd.read_csv('../data/mortality_val.csv'),pd.read_csv('../data/mortality_test.csv')])

In [2]:
# all_data[all_data['label']==0]
pd.read_csv('../data/mortality_val.csv')['label'].sum()

80

In [3]:
all_data['label'].sum()

1688

In [8]:
all_data[all_data['label']==1][['TEXT']].to_csv('../concept_listfiles/mort_30day')

In [9]:
all_data

,Unnamed: 0,TEXT,label
0,1659827,Admission Date: [**2109-12-20**] ...,0
1,672545,"Name: [**Known lastname 8577**], [**Known fir...",0
2,413046,Admission Date: [**2158-3-31**] ...,1
3,291370,Admission Date: [**2126-6-23**] ...,0
4,1309903,Admission Date: [**2161-3-9**] D...,0
...,...,...,...
2345,1320243,Admission Date: [**2107-10-6**] ...,0
2346,1602263,Admission Date: [**2195-8-18**] Dischar...,0
2347,1001916,Admission Date: [**2129-11-13**] ...,0
2348,502531,Admission Date: [**2131-9-28**] ...,0


In [332]:
bins = negative_admission_notes['age_quantile'].unique()
all_quantiles = []
for quantile in bins:
    print(quantile)
    all_quantiles.append(negative_admission_notes[negative_admission_notes['age_quantile']==quantile].sample(negative_quantile_size))

(70.592, 76.059]
(25.656, 57.714]
(81.488, 89.056]
(76.059, 81.488]
(57.714, 64.793]
(64.793, 70.592]


In [353]:
shuffled_age_gender_negatives.groupby('age_quantile').apply(len)

age_quantile
(25.656, 57.714]    1276
(57.714, 64.793]    1272
(64.793, 70.592]    1236
(70.592, 76.059]    1239
(76.059, 81.488]    1196
(81.488, 89.056]    1157
dtype: int64

In [312]:
bins = [0]+bins+[999]

In [303]:
cad_admission_notes.groupby('GENDER').apply(len)

GENDER
F    2316
M    4840
dtype: int64

In [ ]:
def get_brat_sections():
    sections_dict = {}
    sectionspath = '../../LHC_mimic/mimic3_1.4/notes/sections/'
    for hashdir in os.listdir(sectionspath):
        for subjectdir in os.listdir(sectionspath+hashdir):
            for admissiondir in os.listdir(sectionspath+hashdir+'/'+subjectdir):
                for fn in os.listdir(sectionspath+hashdir+'/'+subjectdir+'/'+admissiondir):
                    ann_fn = sectionspath+hashdir+'/'+subjectdir+'/'+admissiondir + '/' + fn
#                     entities, relations, attributes, groups = get_entities_relations_attributes_groups(ann_fn)
#                     with gzip.open(ann_fn, 'rb') as ann_file:
#                         lines = ann_file.readlines()
                    lines= list(pd.read_csv(ann_fn, sep='\n', names=['lines'])['lines'])
                    for line in lines:
                        if line[0]=='A':
                            section=line.split(maxsplit=3)[3]
                            if section in sections_dict.keys():
                                sections_dict[section] = sections_dict[section]+[admissiondir]
                            else: 
                                sections_dict[section] = [admissiondir]
    return sections_dict
brat_sections = get_brat_sections()
sections_df = pd.DataFrame(list(brat_sections.values())).transpose()
sections_df.columns=brat_sections.keys()
sections_df.to_csv('sections_admissions_list.csv')

In [14]:
notes_df['CHARTTIME'] = pd.to_datetime(notes_df['CHARTTIME']) 
notes_df['STORETIME'] = pd.to_datetime(notes_df['STORETIME']) 

In [15]:
admissions_dict = dict(zip(admissions_df['HADM_ID'],admissions_df['ADMITTIME']))

In [16]:
notes_df = notes_df.dropna(subset=['HADM_ID','SUBJECT_ID', 'CHARTTIME', 'TEXT'], how='any')

In [17]:
notes_df['admittime'] = notes_df['HADM_ID'].apply(lambda x: admissions_dict[x] )
notes_df['admittime'] = pd.to_datetime(notes_df['admittime']) 

In [18]:
# after_admission = notes_df.apply(lambda row: row['CHARTTIME']>=row['admittime'], axis=1)

In [19]:
# notes_df = notes_df[after_admission]

In [41]:
# first_cad_notes = cad_notes.sort_values(by='CHARTTIME', ascending=True).drop_duplicates(subset=['HADM_ID'],keep='first')

In [40]:
negative_examples = notes_df[notes_df.apply(lambda row: row['HADM_ID'] not in cad_admissions and row['SUBJECT_ID'] not in cad_patients,axis=1)]

In [36]:
cad_procedure_admissions = procedures_df[procedures_df['ICD9_CODE'].apply(lambda x: x in cad_procedure_codes )]['HADM_ID'].unique()

In [64]:
len(cad_admissions)

13999

In [42]:
cad_procedure_patients = procedures_df[procedures_df['ICD9_CODE'].apply(lambda x: x in cad_procedure_codes )]['SUBJECT_ID'].unique()

In [43]:
negative_examples_minus_cad_procedures = negative_examples[negative_examples.apply(lambda x: x['HADM_ID'] not in cad_procedure_admissions and x['SUBJECT_ID'] not in cad_procedure_patients,axis=1)]

In [65]:
len(negative_examples['HADM_ID'].unique())

39664

In [54]:
len(negative_examples_minus_cad_procedures['HADM_ID'].unique())

19574

In [ ]:
#44 CCS
#neonates
#age, gender distributions
#take first admission of patient use the admission with CAD code

In [46]:
cad_procedures = []
#https://www.aafp.org/afp/2011/0401/p819.html
beta_blockers = [x.lower() for x in  ['Atenolol','Tenormin','Bisoprolol','Zebeta', 'Carvedilol','Coreg', 'Labetalol', 'Metoprolol', 'Propranolol','Inderal']]
anti_platelet = ['aspirin','clopidogrel','plavix' ]
statins = [x.lower() for x in ['Atorvastatin', 'Lipitor', 'Lovastatin', 'Altoprev', 'Pitavastatin', 'Livalo', 'Zypitamag', 'Pravastatin', 'Pravachol','Rosuvastatin','Crestor', 'Ezallor','Simvastatin', 'Zocor']]

In [47]:
# with_aspirin = drugs_df[drugs_df['DRUG'].apply(lambda x: 'aspirin' in x.lower())].groupby('HADM_ID').apply(lambda x: str(x.name) if len(x)>=1 else np.nan)

with_beta_blockers = drugs_df[drugs_df.apply(lambda row: any([x in (str(row['DRUG'])+str(row['DRUG_NAME_POE'])+str(row['DRUG_NAME_GENERIC'])).lower() for x in beta_blockers]) , axis=1)].groupby('SUBJECT_ID').apply(lambda x: str(x.name) if len(x)>=1 else np.nan)

with_anti_platelet = drugs_df[drugs_df.apply(lambda row: any([x in (str(row['DRUG'])+str(row['DRUG_NAME_POE'])+str(row['DRUG_NAME_GENERIC'])).lower() for x in anti_platelet]) , axis=1)].groupby('SUBJECT_ID').apply(lambda x: str(x.name) if len(x)>=1 else np.nan)

with_statins = drugs_df[drugs_df.apply(lambda row: any([x in (str(row['DRUG'])+str(row['DRUG_NAME_POE'])+str(row['DRUG_NAME_GENERIC'])).lower() for x in statins]) , axis=1)].groupby('SUBJECT_ID').apply(lambda x: str(x.name) if len(x)>=1 else np.nan)

In [48]:
all_criteria = set.intersection( set(with_beta_blockers),set(with_anti_platelet),set(with_statins))#,set([str(x) for x in no_coronary_in_drug_description]) )

In [56]:
no_meds_or_procedures = set(negative_examples_minus_cad_procedures['HADM_ID'])-all_criteria

In [51]:
negative_without_cad_meds = negative_examples[negative_examples['HADM_ID'].apply(lambda x: x not in all_criteria)]

In [53]:
len(negative_without_cad_meds['HADM_ID'].unique())

39664

In [1]:
import pandas as pd

In [6]:
test = pd.read_csv('data/CAD_test.csv')

In [10]:
test[test['label']==1][['TEXT']].to_csv('concept_listfiles/CAD_final')